# 勾配報酬スケーリングの効果検証実験

この実験では、勾配ベース報酬に対してL_hatスケーリングを適用する効果を検証します：

1. **スケーリングあり**: L_hatスケーリング + 勾配ベース報酬 + 0.5x arms
2. **スケーリングなし**: 勾配ベース報酬（スケーリングなし） + 0.5x arms

両アルゴリズムは以下の共通設定を使用：
- 勾配ベース報酬（GPモデルの勾配の絶対値）
- 0.5x arms（次元数の半分のアーム数）
- coordinate_ratio=0.8

テスト関数：
- Styblinski-Tang
- Rastrigin
- Ackley

各関数について20回の独立実行を行い、L_hatスケーリングの純粋な効果を評価します。

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import torch
import os
from copy import deepcopy
import pandas as pd

# BoTorch / GPyTorch
from botorch import fit_gpytorch_model
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from gpytorch.kernels import RBFKernel, ScaleKernel
from botorch.acquisition import ExpectedImprovement
from botorch.optim import optimize_acqf

# デフォルトのdtypeをfloat32に設定
torch.set_default_dtype(torch.float32)

# プロット設定
plt.rcParams["figure.dpi"] = 100
plt.rcParams['figure.figsize'] = (12, 8)

# 日本語フォント設定
# 方法1: japanize-matplotlibを試す
try:
    import japanize_matplotlib
except ImportError:
    # 方法2: 手動でフォントを設定
    import matplotlib
    # Windowsの場合
    if os.name == 'nt':
        plt.rcParams['font.family'] = ['MS Gothic', 'Yu Gothic', 'Meiryo']
    # macOSの場合
    elif os.uname().sysname == 'Darwin':
        plt.rcParams['font.family'] = ['Hiragino Sans', 'Hiragino Maru Gothic Pro']
    # Linuxの場合
    else:
        plt.rcParams['font.family'] = ['IPAGothic', 'IPAPGothic', 'VL PGothic', 'Noto Sans CJK JP', 'TakaoGothic']
    
    plt.rcParams['axes.unicode_minus'] = False

# フォント設定の確認
print(f"使用フォント: {plt.rcParams['font.family']}")

import warnings
warnings.filterwarnings("ignore")

# 出力フォルダの作成
output_dir = "output_results_gradient_scaling_comparison"
os.makedirs(output_dir, exist_ok=True)

print("実験環境の設定完了")

In [ ]:
# スケーリングあり版LinBandit-BO（固定L_hatスケーリング + 勾配ベース + 0.5x arms）
class GradientLinBanditBO_WithFixedScaling:
    """
    勾配報酬 + 固定L_hatスケーリングあり版LinBandit-BO
    """
    
    def __init__(self, objective_function, bounds, n_initial=5, n_max=100, 
                 coordinate_ratio=0.8, n_arms=None, fixed_L_hat=100.0):
        self.objective_function = objective_function
        self.bounds = bounds.float()
        self.dim = bounds.shape[1]
        self.n_initial = n_initial
        self.n_max = n_max
        self.coordinate_ratio = coordinate_ratio
        
        # 0.5x arms設定
        self.n_arms = n_arms if n_arms is not None else max(1, self.dim // 2)
        
        # Linear Banditのパラメータ
        self.A = torch.eye(self.dim)
        self.b = torch.zeros(self.dim)
        
        # 初期点の生成
        self.X = torch.rand(n_initial, self.dim) * (bounds[1] - bounds[0]) + bounds[0]
        self.X = self.X.float()
        
        # 状態変数
        self.Y = None
        self.best_value = None
        self.best_point = None
        self.model = None
        self.eval_history = []
        self.theta_history = []
        self.scale_init = 1.0
        self.total_iterations = 0
        
        # 固定リプシッツ定数
        self.L_hat = fixed_L_hat
        
        # 報酬履歴の記録
        self.reward_history = []
        self.scaled_reward_history = []
        
    def update_model(self):
        """ガウス過程モデルの更新"""
        kernel = ScaleKernel(
            RBFKernel(ard_num_dims=self.X.shape[-1], dtype=torch.float32),
            dtype=torch.float32, noise_constraint=1e-3
        ).to(self.X)
        self.model = SingleTaskGP(self.X, self.Y, covar_module=kernel)
        mll = ExactMarginalLogLikelihood(self.model.likelihood, self.model)
        fit_gpytorch_model(mll)
        
    def initialize(self):
        """初期化：初期点での評価とモデル構築"""
        y_val = self.objective_function(self.X)
        self.Y = y_val.unsqueeze(-1).float()
        
        # スケーリング係数の計算
        y_max, y_min = self.Y.max().item(), self.Y.min().item()
        self.scale_init = (y_max - y_min) if (y_max - y_min) != 0 else 1.0
        
        # モデルの初期化
        self.update_model()
        
        # 最良点の初期化
        post_mean = self.model.posterior(self.X).mean.squeeze(-1)
        bi = post_mean.argmin()
        self.best_value = post_mean[bi].item()
        self.best_point = self.X[bi]
        self.eval_history = [self.best_value] * self.n_initial
        
    def generate_arms(self):
        """アーム生成（0.5x arms）"""
        num_coord = int(self.coordinate_ratio * self.n_arms)
        num_coord = min(num_coord, self.dim)
        
        # ランダムに座標を選択
        idxs = np.random.choice(self.dim, num_coord, replace=False)
        
        # 座標方向の生成
        coords = []
        for i in idxs:
            e = torch.zeros(self.dim, device=self.X.device)
            e[i] = 1.0
            coords.append(e)
            
        coord_arms = torch.stack(coords, 0) if coords else torch.zeros(0, self.dim, device=self.X.device)
        
        # ランダム方向の生成
        num_rand = self.n_arms - num_coord
        rand_arms = torch.randn(num_rand, self.dim, device=self.X.device) if num_rand > 0 else torch.zeros(0, self.dim, device=self.X.device)
        
        if num_rand > 0:
            norms = rand_arms.norm(dim=1, keepdim=True)
            rand_arms = torch.where(norms > 1e-9, rand_arms / norms, 
                                   torch.randn_like(rand_arms) / (torch.randn_like(rand_arms).norm(dim=1,keepdim=True)+1e-9))
            
        return torch.cat([coord_arms, rand_arms], 0)
    
    def select_arm(self, arms_features):
        """Linear UCBによる方向選択"""
        # LinUCBパラメータ
        sigma = 1.0
        L = 1.0
        lambda_reg = 1.0
        delta = 0.1
        S = 1.0
        
        # 現在のパラメータ推定
        A_inv = torch.inverse(self.A)
        theta = A_inv @ self.b
        self.theta_history.append(theta.clone())
        
        # 信頼幅の計算
        current_round_t = max(1, self.total_iterations)
        log_term_numerator = max(1e-9, 1 + (current_round_t - 1) * L**2 / lambda_reg)
        beta_t = (sigma * math.sqrt(self.dim * math.log(log_term_numerator / delta)) + 
                  math.sqrt(lambda_reg) * S)
        
        # UCBスコアの計算
        ucb_scores = []
        for i in range(arms_features.shape[0]):
            x = arms_features[i].view(-1, 1)
            mean = (theta.view(1, -1) @ x).item()
            try:
                var = (x.t() @ A_inv @ x).item()
            except torch.linalg.LinAlgError:
                var = (x.t() @ torch.linalg.pinv(self.A) @ x).item()
                
            ucb_scores.append(mean + beta_t * math.sqrt(max(var, 0)))
            
        return int(np.argmax(ucb_scores))
    
    def propose_new_x(self, direction):
        """選択された方向に沿った最適化"""
        ei = ExpectedImprovement(self.model, best_f=self.best_value, maximize=False)
        
        # 方向に沿った探索範囲の計算
        active_dims_mask = direction.abs() > 1e-9
        if not active_dims_mask.any():
            lb, ub = -1.0, 1.0
        else:
            ratios_lower = (self.bounds[0] - self.best_point) / (direction + 1e-12 * (~active_dims_mask))
            ratios_upper = (self.bounds[1] - self.best_point) / (direction + 1e-12 * (~active_dims_mask))
            
            t_bounds = torch.zeros(self.dim, 2, device=self.X.device)
            t_bounds[:, 0] = torch.minimum(ratios_lower, ratios_upper)
            t_bounds[:, 1] = torch.maximum(ratios_lower, ratios_upper)
            
            lb = -float('inf')
            ub = float('inf')
            for i in range(self.dim):
                if active_dims_mask[i]:
                    lb = max(lb, t_bounds[i, 0].item())
                    ub = min(ub, t_bounds[i, 1].item())
                    
        if lb > ub:
            domain_width = (self.bounds[1, 0] - self.bounds[0, 0]).item()
            lb = -0.1 * domain_width
            ub = 0.1 * domain_width
            
        one_d_bounds = torch.tensor([[lb], [ub]], dtype=torch.float32, device=self.X.device)
        
        def ei_on_line(t_scalar_tensor):
            t_values = t_scalar_tensor.squeeze(-1)
            points_on_line = self.best_point.unsqueeze(0) + t_values.reshape(-1, 1) * direction.unsqueeze(0)
            points_on_line_clamped = torch.clamp(points_on_line, self.bounds[0].unsqueeze(0), self.bounds[1].unsqueeze(0))
            return ei(points_on_line_clamped.unsqueeze(1))
        
        # 獲得関数の最適化
        cand_t, _ = optimize_acqf(
            ei_on_line,
            bounds=one_d_bounds,
            q=1,
            num_restarts=10,
            raw_samples=100
        )
        
        alpha_star = cand_t.item()
        new_x = self.best_point + alpha_star * direction
        new_x_clamped = torch.clamp(new_x, self.bounds[0], self.bounds[1])
        
        return new_x_clamped
    
    def optimize(self):
        """メインの最適化ループ"""
        # 初期化
        self.initialize()
        n_iter = self.n_initial
        
        while n_iter < self.n_max:
            self.total_iterations += 1
            
            # 探索方向の候補生成
            arms_features = self.generate_arms()
            
            # Linear UCBによる方向選択
            sel_idx = self.select_arm(arms_features)
            direction = arms_features[sel_idx]
            
            # 選択された方向に沿った最適化
            new_x = self.propose_new_x(direction)
            
            # 予測と実際の評価
            with torch.no_grad():
                predicted_mean = self.model.posterior(new_x.unsqueeze(0)).mean.squeeze().item()
            actual_y = self.objective_function(new_x.unsqueeze(0)).squeeze().item()
            
            # 報酬の計算（勾配ベース + 固定L_hatスケーリング）
            new_x_for_grad = new_x.clone().unsqueeze(0)
            new_x_for_grad.requires_grad_(True)
            
            # GPモデルで事後分布を取得
            posterior = self.model.posterior(new_x_for_grad)
            mean_at_new_x = posterior.mean
            
            # 勾配を計算
            mean_at_new_x.sum().backward()
            grad_vector = new_x_for_grad.grad.squeeze(0)
            
            # 報酬ベクトルを定義
            reward_vector = grad_vector.abs()
            
            # 報酬を記録
            self.reward_history.append(reward_vector.clone().detach().cpu().numpy())
            
            # 固定リプシッツ定数でスケーリング
            scaled_reward_vector = reward_vector / self.L_hat
            
            # スケーリングされた報酬を記録
            self.scaled_reward_history.append(scaled_reward_vector.clone().detach().cpu().numpy())
            
            # Linear Banditパラメータの更新
            x_arm = direction.view(-1, 1)
            self.A += x_arm @ x_arm.t()
            self.b += scaled_reward_vector
            
            # データとモデルの更新
            self.X = torch.cat([self.X, new_x.unsqueeze(0)], 0)
            self.Y = torch.cat([self.Y, torch.tensor([[actual_y]], dtype=torch.float32, device=self.X.device)], 0)
            self.update_model()
            
            # 最良点の更新
            with torch.no_grad():
                posterior_mean = self.model.posterior(self.X).mean.squeeze(-1)
            current_best_idx = posterior_mean.argmin()
            self.best_value = posterior_mean[current_best_idx].item()
            self.best_point = self.X[current_best_idx]
            
            self.eval_history.append(self.best_value)
            n_iter += 1
                
        return self.best_point, self.best_value

print("固定スケーリングあり版LinBandit-BOクラスの定義完了")

In [10]:
# スケーリングあり版LinBandit-BO（L_hatスケーリング + 勾配ベース + 0.5x arms）
class GradientLinBanditBO_WithScaling:
    """
    勾配報酬 + L_hatスケーリングあり版LinBandit-BO
    """
    
    def __init__(self, objective_function, bounds, n_initial=5, n_max=100, 
                 coordinate_ratio=0.8, n_arms=None):
        self.objective_function = objective_function
        self.bounds = bounds.float()
        self.dim = bounds.shape[1]
        self.n_initial = n_initial
        self.n_max = n_max
        self.coordinate_ratio = coordinate_ratio
        
        # 0.5x arms設定
        self.n_arms = n_arms if n_arms is not None else max(1, self.dim // 2)
        
        # Linear Banditのパラメータ
        self.A = torch.eye(self.dim)
        self.b = torch.zeros(self.dim)
        
        # 初期点の生成
        self.X = torch.rand(n_initial, self.dim) * (bounds[1] - bounds[0]) + bounds[0]
        self.X = self.X.float()
        
        # 状態変数
        self.Y = None
        self.best_value = None
        self.best_point = None
        self.model = None
        self.eval_history = []
        self.theta_history = []
        self.scale_init = 1.0
        self.total_iterations = 0
        
        # 推定リプシッツ定数（スケーリングあり）
        self.L_hat = 1.0
        
    def update_model(self):
        """ガウス過程モデルの更新"""
        kernel = ScaleKernel(
            RBFKernel(ard_num_dims=self.X.shape[-1], dtype=torch.float32),
            dtype=torch.float32, noise_constraint=1e-3
        ).to(self.X)
        self.model = SingleTaskGP(self.X, self.Y, covar_module=kernel)
        mll = ExactMarginalLogLikelihood(self.model.likelihood, self.model)
        fit_gpytorch_model(mll)
        
    def initialize(self):
        """初期化：初期点での評価とモデル構築"""
        y_val = self.objective_function(self.X)
        self.Y = y_val.unsqueeze(-1).float()
        
        # スケーリング係数の計算
        y_max, y_min = self.Y.max().item(), self.Y.min().item()
        self.scale_init = (y_max - y_min) if (y_max - y_min) != 0 else 1.0
        
        # モデルの初期化
        self.update_model()
        
        # 最良点の初期化
        post_mean = self.model.posterior(self.X).mean.squeeze(-1)
        bi = post_mean.argmin()
        self.best_value = post_mean[bi].item()
        self.best_point = self.X[bi]
        self.eval_history = [self.best_value] * self.n_initial
        
    def generate_arms(self):
        """アーム生成（0.5x arms）"""
        num_coord = int(self.coordinate_ratio * self.n_arms)
        num_coord = min(num_coord, self.dim)
        
        # ランダムに座標を選択
        idxs = np.random.choice(self.dim, num_coord, replace=False)
        
        # 座標方向の生成
        coords = []
        for i in idxs:
            e = torch.zeros(self.dim, device=self.X.device)
            e[i] = 1.0
            coords.append(e)
            
        coord_arms = torch.stack(coords, 0) if coords else torch.zeros(0, self.dim, device=self.X.device)
        
        # ランダム方向の生成
        num_rand = self.n_arms - num_coord
        rand_arms = torch.randn(num_rand, self.dim, device=self.X.device) if num_rand > 0 else torch.zeros(0, self.dim, device=self.X.device)
        
        if num_rand > 0:
            norms = rand_arms.norm(dim=1, keepdim=True)
            rand_arms = torch.where(norms > 1e-9, rand_arms / norms, 
                                   torch.randn_like(rand_arms) / (torch.randn_like(rand_arms).norm(dim=1,keepdim=True)+1e-9))
            
        return torch.cat([coord_arms, rand_arms], 0)
    
    def select_arm(self, arms_features):
        """Linear UCBによる方向選択"""
        # LinUCBパラメータ
        sigma = 1.0
        L = 1.0
        lambda_reg = 1.0
        delta = 0.1
        S = 1.0
        
        # 現在のパラメータ推定
        A_inv = torch.inverse(self.A)
        theta = A_inv @ self.b
        self.theta_history.append(theta.clone())
        
        # 信頼幅の計算
        current_round_t = max(1, self.total_iterations)
        log_term_numerator = max(1e-9, 1 + (current_round_t - 1) * L**2 / lambda_reg)
        beta_t = (sigma * math.sqrt(self.dim * math.log(log_term_numerator / delta)) + 
                  math.sqrt(lambda_reg) * S)
        
        # UCBスコアの計算
        ucb_scores = []
        for i in range(arms_features.shape[0]):
            x = arms_features[i].view(-1, 1)
            mean = (theta.view(1, -1) @ x).item()
            try:
                var = (x.t() @ A_inv @ x).item()
            except torch.linalg.LinAlgError:
                var = (x.t() @ torch.linalg.pinv(self.A) @ x).item()
                
            ucb_scores.append(mean + beta_t * math.sqrt(max(var, 0)))
            
        return int(np.argmax(ucb_scores))
    
    def propose_new_x(self, direction):
        """選択された方向に沿った最適化"""
        ei = ExpectedImprovement(self.model, best_f=self.best_value, maximize=False)
        
        # 方向に沿った探索範囲の計算
        active_dims_mask = direction.abs() > 1e-9
        if not active_dims_mask.any():
            lb, ub = -1.0, 1.0
        else:
            ratios_lower = (self.bounds[0] - self.best_point) / (direction + 1e-12 * (~active_dims_mask))
            ratios_upper = (self.bounds[1] - self.best_point) / (direction + 1e-12 * (~active_dims_mask))
            
            t_bounds = torch.zeros(self.dim, 2, device=self.X.device)
            t_bounds[:, 0] = torch.minimum(ratios_lower, ratios_upper)
            t_bounds[:, 1] = torch.maximum(ratios_lower, ratios_upper)
            
            lb = -float('inf')
            ub = float('inf')
            for i in range(self.dim):
                if active_dims_mask[i]:
                    lb = max(lb, t_bounds[i, 0].item())
                    ub = min(ub, t_bounds[i, 1].item())
                    
        if lb > ub:
            domain_width = (self.bounds[1, 0] - self.bounds[0, 0]).item()
            lb = -0.1 * domain_width
            ub = 0.1 * domain_width
            
        one_d_bounds = torch.tensor([[lb], [ub]], dtype=torch.float32, device=self.X.device)
        
        def ei_on_line(t_scalar_tensor):
            t_values = t_scalar_tensor.squeeze(-1)
            points_on_line = self.best_point.unsqueeze(0) + t_values.reshape(-1, 1) * direction.unsqueeze(0)
            points_on_line_clamped = torch.clamp(points_on_line, self.bounds[0].unsqueeze(0), self.bounds[1].unsqueeze(0))
            return ei(points_on_line_clamped.unsqueeze(1))
        
        # 獲得関数の最適化
        cand_t, _ = optimize_acqf(
            ei_on_line,
            bounds=one_d_bounds,
            q=1,
            num_restarts=10,
            raw_samples=100
        )
        
        alpha_star = cand_t.item()
        new_x = self.best_point + alpha_star * direction
        new_x_clamped = torch.clamp(new_x, self.bounds[0], self.bounds[1])
        
        return new_x_clamped
    
    def optimize(self):
        """メインの最適化ループ"""
        # 初期化
        self.initialize()
        n_iter = self.n_initial
        
        while n_iter < self.n_max:
            self.total_iterations += 1
            
            # 探索方向の候補生成
            arms_features = self.generate_arms()
            
            # Linear UCBによる方向選択
            sel_idx = self.select_arm(arms_features)
            direction = arms_features[sel_idx]
            
            # 選択された方向に沿った最適化
            new_x = self.propose_new_x(direction)
            
            # 予測と実際の評価
            with torch.no_grad():
                predicted_mean = self.model.posterior(new_x.unsqueeze(0)).mean.squeeze().item()
            actual_y = self.objective_function(new_x.unsqueeze(0)).squeeze().item()
            
            # 報酬の計算（勾配ベース + L_hatスケーリング）
            new_x_for_grad = new_x.clone().unsqueeze(0)
            new_x_for_grad.requires_grad_(True)
            
            # GPモデルで事後分布を取得
            posterior = self.model.posterior(new_x_for_grad)
            mean_at_new_x = posterior.mean
            
            # 勾配を計算
            mean_at_new_x.sum().backward()
            grad_vector = new_x_for_grad.grad.squeeze(0)
            
            # 報酬ベクトルを定義
            reward_vector = grad_vector.abs()
            
            # 推定リプシッツ定数の更新
            grad_norm = reward_vector.norm().item()
            if grad_norm > self.L_hat:
                self.L_hat = grad_norm
            
            # リプシッツ定数でスケーリング（スケーリングあり）
            scaled_reward_vector = reward_vector / self.L_hat
            
            # Linear Banditパラメータの更新
            x_arm = direction.view(-1, 1)
            self.A += x_arm @ x_arm.t()
            self.b += scaled_reward_vector
            
            # データとモデルの更新
            self.X = torch.cat([self.X, new_x.unsqueeze(0)], 0)
            self.Y = torch.cat([self.Y, torch.tensor([[actual_y]], dtype=torch.float32, device=self.X.device)], 0)
            self.update_model()
            
            # 最良点の更新
            with torch.no_grad():
                posterior_mean = self.model.posterior(self.X).mean.squeeze(-1)
            current_best_idx = posterior_mean.argmin()
            self.best_value = posterior_mean[current_best_idx].item()
            self.best_point = self.X[current_best_idx]
            
            self.eval_history.append(self.best_value)
            n_iter += 1
                
        return self.best_point, self.best_value

print("スケーリングあり版LinBandit-BOクラスの定義完了")

スケーリングあり版LinBandit-BOクラスの定義完了


In [ ]:
# スケーリングなし版LinBandit-BO（勾配ベース + 0.5x arms、スケーリングなし）
class GradientLinBanditBO_WithoutScaling:
    """
    勾配報酬 + L_hatスケーリングなし版LinBandit-BO
    """
    
    def __init__(self, objective_function, bounds, n_initial=5, n_max=100, 
                 coordinate_ratio=0.8, n_arms=None):
        self.objective_function = objective_function
        self.bounds = bounds.float()
        self.dim = bounds.shape[1]
        self.n_initial = n_initial
        self.n_max = n_max
        self.coordinate_ratio = coordinate_ratio
        
        # 0.5x arms設定
        self.n_arms = n_arms if n_arms is not None else max(1, self.dim // 2)
        
        # Linear Banditのパラメータ
        self.A = torch.eye(self.dim)
        self.b = torch.zeros(self.dim)
        
        # 初期点の生成
        self.X = torch.rand(n_initial, self.dim) * (bounds[1] - bounds[0]) + bounds[0]
        self.X = self.X.float()
        
        # 状態変数
        self.Y = None
        self.best_value = None
        self.best_point = None
        self.model = None
        self.eval_history = []
        self.theta_history = []
        self.scale_init = 1.0
        self.total_iterations = 0
        
        # 報酬履歴の記録
        self.reward_history = []
        
    def update_model(self):
        """ガウス過程モデルの更新"""
        kernel = ScaleKernel(
            RBFKernel(ard_num_dims=self.X.shape[-1], dtype=torch.float32),
            dtype=torch.float32, noise_constraint=1e-3
        ).to(self.X)
        self.model = SingleTaskGP(self.X, self.Y, covar_module=kernel)
        mll = ExactMarginalLogLikelihood(self.model.likelihood, self.model)
        fit_gpytorch_model(mll)
        
    def initialize(self):
        """初期化：初期点での評価とモデル構築"""
        y_val = self.objective_function(self.X)
        self.Y = y_val.unsqueeze(-1).float()
        
        # スケーリング係数の計算
        y_max, y_min = self.Y.max().item(), self.Y.min().item()
        self.scale_init = (y_max - y_min) if (y_max - y_min) != 0 else 1.0
        
        # モデルの初期化
        self.update_model()
        
        # 最良点の初期化
        post_mean = self.model.posterior(self.X).mean.squeeze(-1)
        bi = post_mean.argmin()
        self.best_value = post_mean[bi].item()
        self.best_point = self.X[bi]
        self.eval_history = [self.best_value] * self.n_initial
        
    def generate_arms(self):
        """アーム生成（0.5x arms）"""
        num_coord = int(self.coordinate_ratio * self.n_arms)
        num_coord = min(num_coord, self.dim)
        
        # ランダムに座標を選択
        idxs = np.random.choice(self.dim, num_coord, replace=False)
        
        # 座標方向の生成
        coords = []
        for i in idxs:
            e = torch.zeros(self.dim, device=self.X.device)
            e[i] = 1.0
            coords.append(e)
            
        coord_arms = torch.stack(coords, 0) if coords else torch.zeros(0, self.dim, device=self.X.device)
        
        # ランダム方向の生成
        num_rand = self.n_arms - num_coord
        rand_arms = torch.randn(num_rand, self.dim, device=self.X.device) if num_rand > 0 else torch.zeros(0, self.dim, device=self.X.device)
        
        if num_rand > 0:
            norms = rand_arms.norm(dim=1, keepdim=True)
            rand_arms = torch.where(norms > 1e-9, rand_arms / norms, 
                                   torch.randn_like(rand_arms) / (torch.randn_like(rand_arms).norm(dim=1,keepdim=True)+1e-9))
            
        return torch.cat([coord_arms, rand_arms], 0)
    
    def select_arm(self, arms_features):
        """Linear UCBによる方向選択"""
        # LinUCBパラメータ
        sigma = 1.0
        L = 1.0
        lambda_reg = 1.0
        delta = 0.1
        S = 1.0
        
        # 現在のパラメータ推定
        A_inv = torch.inverse(self.A)
        theta = A_inv @ self.b
        self.theta_history.append(theta.clone())
        
        # 信頼幅の計算
        current_round_t = max(1, self.total_iterations)
        log_term_numerator = max(1e-9, 1 + (current_round_t - 1) * L**2 / lambda_reg)
        beta_t = (sigma * math.sqrt(self.dim * math.log(log_term_numerator / delta)) + 
                  math.sqrt(lambda_reg) * S)
        
        # UCBスコアの計算
        ucb_scores = []
        for i in range(arms_features.shape[0]):
            x = arms_features[i].view(-1, 1)
            mean = (theta.view(1, -1) @ x).item()
            try:
                var = (x.t() @ A_inv @ x).item()
            except torch.linalg.LinAlgError:
                var = (x.t() @ torch.linalg.pinv(self.A) @ x).item()
                
            ucb_scores.append(mean + beta_t * math.sqrt(max(var, 0)))
            
        return int(np.argmax(ucb_scores))
    
    def propose_new_x(self, direction):
        """選択された方向に沿った最適化"""
        ei = ExpectedImprovement(self.model, best_f=self.best_value, maximize=False)
        
        # 方向に沿った探索範囲の計算
        active_dims_mask = direction.abs() > 1e-9
        if not active_dims_mask.any():
            lb, ub = -1.0, 1.0
        else:
            ratios_lower = (self.bounds[0] - self.best_point) / (direction + 1e-12 * (~active_dims_mask))
            ratios_upper = (self.bounds[1] - self.best_point) / (direction + 1e-12 * (~active_dims_mask))
            
            t_bounds = torch.zeros(self.dim, 2, device=self.X.device)
            t_bounds[:, 0] = torch.minimum(ratios_lower, ratios_upper)
            t_bounds[:, 1] = torch.maximum(ratios_lower, ratios_upper)
            
            lb = -float('inf')
            ub = float('inf')
            for i in range(self.dim):
                if active_dims_mask[i]:
                    lb = max(lb, t_bounds[i, 0].item())
                    ub = min(ub, t_bounds[i, 1].item())
                    
        if lb > ub:
            domain_width = (self.bounds[1, 0] - self.bounds[0, 0]).item()
            lb = -0.1 * domain_width
            ub = 0.1 * domain_width
            
        one_d_bounds = torch.tensor([[lb], [ub]], dtype=torch.float32, device=self.X.device)
        
        def ei_on_line(t_scalar_tensor):
            t_values = t_scalar_tensor.squeeze(-1)
            points_on_line = self.best_point.unsqueeze(0) + t_values.reshape(-1, 1) * direction.unsqueeze(0)
            points_on_line_clamped = torch.clamp(points_on_line, self.bounds[0].unsqueeze(0), self.bounds[1].unsqueeze(0))
            return ei(points_on_line_clamped.unsqueeze(1))
        
        # 獲得関数の最適化
        cand_t, _ = optimize_acqf(
            ei_on_line,
            bounds=one_d_bounds,
            q=1,
            num_restarts=10,
            raw_samples=100
        )
        
        alpha_star = cand_t.item()
        new_x = self.best_point + alpha_star * direction
        new_x_clamped = torch.clamp(new_x, self.bounds[0], self.bounds[1])
        
        return new_x_clamped
    
    def optimize(self):
        """メインの最適化ループ"""
        # 初期化
        self.initialize()
        n_iter = self.n_initial
        
        while n_iter < self.n_max:
            self.total_iterations += 1
            
            # 探索方向の候補生成
            arms_features = self.generate_arms()
            
            # Linear UCBによる方向選択
            sel_idx = self.select_arm(arms_features)
            direction = arms_features[sel_idx]
            
            # 選択された方向に沿った最適化
            new_x = self.propose_new_x(direction)
            
            # 予測と実際の評価
            with torch.no_grad():
                predicted_mean = self.model.posterior(new_x.unsqueeze(0)).mean.squeeze().item()
            actual_y = self.objective_function(new_x.unsqueeze(0)).squeeze().item()
            
            # 報酬の計算（勾配ベース、スケーリングなし）
            new_x_for_grad = new_x.clone().unsqueeze(0)
            new_x_for_grad.requires_grad_(True)
            
            # GPモデルで事後分布を取得
            posterior = self.model.posterior(new_x_for_grad)
            mean_at_new_x = posterior.mean
            
            # 勾配を計算
            mean_at_new_x.sum().backward()
            grad_vector = new_x_for_grad.grad.squeeze(0)
            
            # 報酬ベクトルを定義（スケーリングなし）
            reward_vector = grad_vector.abs()
            
            # 報酬を記録
            self.reward_history.append(reward_vector.clone().detach().cpu().numpy())
            
            # Linear Banditパラメータの更新（スケーリングなし）
            x_arm = direction.view(-1, 1)
            self.A += x_arm @ x_arm.t()
            self.b += reward_vector  # スケーリングしない
            
            # データとモデルの更新
            self.X = torch.cat([self.X, new_x.unsqueeze(0)], 0)
            self.Y = torch.cat([self.Y, torch.tensor([[actual_y]], dtype=torch.float32, device=self.X.device)], 0)
            self.update_model()
            
            # 最良点の更新
            with torch.no_grad():
                posterior_mean = self.model.posterior(self.X).mean.squeeze(-1)
            current_best_idx = posterior_mean.argmin()
            self.best_value = posterior_mean[current_best_idx].item()
            self.best_point = self.X[current_best_idx]
            
            self.eval_history.append(self.best_value)
            n_iter += 1
                
        return self.best_point, self.best_value

print("スケーリングなし版LinBandit-BOクラスの定義完了")

In [12]:
# テスト関数の定義
def styblinski_tang_effective(x, effective_dims=5):
    """Styblinski-Tang関数（有効次元のみ）"""
    if not torch.is_tensor(x):
        x = torch.tensor(x, dtype=torch.float32)
    
    # 有効次元のみを使用
    x_eff = x[..., :effective_dims]
    return 0.5 * torch.sum(x_eff**4 - 16.0*x_eff**2 + 5.0*x_eff, dim=-1)

def rastrigin_effective(x, effective_dims=5):
    """Rastrigin関数（有効次元のみ）"""
    if not torch.is_tensor(x):
        x = torch.tensor(x, dtype=torch.float32)
    
    # 有効次元のみを使用
    x_eff = x[..., :effective_dims]
    return torch.sum(x_eff**2 - 10.0*torch.cos(2*math.pi*x_eff) + 10.0, dim=-1)

def ackley_effective(x, effective_dims=5):
    """Ackley関数（有効次元のみ）"""
    if not torch.is_tensor(x):
        x = torch.tensor(x, dtype=torch.float32)
    
    # 有効次元のみを使用
    x_eff = x[..., :effective_dims]
    d = x_eff.shape[-1]
    
    sum1 = torch.sum(x_eff**2, dim=-1)
    sum2 = torch.sum(torch.cos(2*math.pi*x_eff), dim=-1)
    
    return -20.0 * torch.exp(-0.2 * torch.sqrt(sum1/d)) - torch.exp(sum2/d) + 20.0 + math.e

# テスト関数の設定
test_functions = {
    'Styblinski-Tang': styblinski_tang_effective,
    'Rastrigin': rastrigin_effective,
    'Ackley': ackley_effective
}

# 大域的最適値
global_optima = {
    'Styblinski-Tang': -39.16599 * 5,  # 5次元
    'Rastrigin': 0.0,
    'Ackley': 0.0
}

print("テスト関数の定義完了")

テスト関数の定義完了


In [13]:
# 実験実行関数
def run_single_experiment(algorithm_class, objective_function, bounds, algorithm_name):
    """単一実験の実行"""
    optimizer = algorithm_class(
        objective_function=objective_function,
        bounds=bounds,
        n_initial=5,
        n_max=300,
        coordinate_ratio=0.8
    )
    
    # 同じ初期点を使用するため、シードを固定
    torch.manual_seed(42)
    np.random.seed(42)
    
    optimizer.optimize()
    
    result = {
        'eval_history': optimizer.eval_history,
        'best_value': optimizer.best_value,
        'theta_history': optimizer.theta_history
    }
    
    # L_hat情報の追加
    if hasattr(optimizer, 'L_hat'):
        result['L_hat'] = optimizer.L_hat
    elif hasattr(optimizer, 'L_hat_record'):
        result['L_hat_record'] = optimizer.L_hat_record
    
    return result

def run_comparison_experiment(func_name, objective_function, n_runs=20):
    """比較実験の実行"""
    print(f"\n=== {func_name} 実験開始 ===")
    
    dim = 20
    bounds = torch.tensor([[-5.0]*dim, [5.0]*dim], dtype=torch.float32)
    
    results = {
        'WithScaling': [],
        'WithoutScaling': []
    }
    
    algorithms = {
        'WithScaling': GradientLinBanditBO_WithScaling,
        'WithoutScaling': GradientLinBanditBO_WithoutScaling
    }
    
    for alg_name, alg_class in algorithms.items():
        print(f"{alg_name}の実験中...")
        for run_idx in range(n_runs):
            print(f"\r  実行中: {run_idx + 1}/{n_runs}", end="", flush=True)
            
            # 各実行で異なるシードを使用
            torch.manual_seed(run_idx * 100)
            np.random.seed(run_idx * 100)
            
            result = run_single_experiment(alg_class, objective_function, bounds, alg_name)
            results[alg_name].append(result)
        
        print(f"\n  {alg_name}完了")
    
    return results

print("実験実行関数の定義完了")

実験実行関数の定義完了


In [ ]:
# 実験実行関数
def run_single_experiment(algorithm_class, objective_function, bounds, algorithm_name, fixed_L_hat=None):
    """単一実験の実行"""
    if algorithm_name == "WithScaling" and fixed_L_hat is not None:
        optimizer = algorithm_class(
            objective_function=objective_function,
            bounds=bounds,
            n_initial=5,
            n_max=300,
            coordinate_ratio=0.8,
            fixed_L_hat=fixed_L_hat
        )
    else:
        optimizer = algorithm_class(
            objective_function=objective_function,
            bounds=bounds,
            n_initial=5,
            n_max=300,
            coordinate_ratio=0.8
        )
    
    # 同じ初期点を使用するため、シードを固定
    torch.manual_seed(42)
    np.random.seed(42)
    
    optimizer.optimize()
    
    result = {
        'eval_history': optimizer.eval_history,
        'best_value': optimizer.best_value,
        'theta_history': optimizer.theta_history,
        'reward_history': optimizer.reward_history
    }
    
    # スケーリングされた報酬履歴も保存（スケーリングありの場合）
    if hasattr(optimizer, 'scaled_reward_history'):
        result['scaled_reward_history'] = optimizer.scaled_reward_history
        result['L_hat'] = optimizer.L_hat
    
    return result

def run_comparison_experiment(func_name, objective_function, analytical_L_hat, n_runs=20):
    """比較実験の実行"""
    print(f"\n=== {func_name} 実験開始 ===")
    print(f"解析的L_hat: {analytical_L_hat:.2f}")
    
    dim = 20
    bounds = torch.tensor([[-5.0]*dim, [5.0]*dim], dtype=torch.float32)
    
    results = {
        'WithScaling': [],
        'WithoutScaling': []
    }
    
    algorithms = {
        'WithScaling': GradientLinBanditBO_WithFixedScaling,
        'WithoutScaling': GradientLinBanditBO_WithoutScaling
    }
    
    for alg_name, alg_class in algorithms.items():
        print(f"{alg_name}の実験中...")
        for run_idx in range(n_runs):
            print(f"\r  実行中: {run_idx + 1}/{n_runs}", end="", flush=True)
            
            # 各実行で異なるシードを使用
            torch.manual_seed(run_idx * 100)
            np.random.seed(run_idx * 100)
            
            result = run_single_experiment(alg_class, objective_function, bounds, alg_name, analytical_L_hat)
            results[alg_name].append(result)
        
        print(f"\n  {alg_name}完了")
    
    return results

print("実験実行関数の定義完了")

In [ ]:
# 報酬履歴の分析関数
def analyze_reward_history(results_dict, func_name):
    """報酬履歴の詳細分析とデータフレーム化"""
    
    # 各実行の報酬履歴を結合
    reward_data = []
    
    for alg_name, results in results_dict.items():
        for run_idx, result in enumerate(results):
            reward_history = result['reward_history']
            
            # 各イテレーションの報酬を記録
            for iter_idx, rewards in enumerate(reward_history):
                for dim_idx, reward in enumerate(rewards):
                    reward_data.append({
                        'Algorithm': alg_name,
                        'Run': run_idx,
                        'Iteration': iter_idx + 5,  # n_initial=5を考慮
                        'Dimension': dim_idx,
                        'Reward': reward,
                        'Scaled_Reward': result['scaled_reward_history'][iter_idx][dim_idx] if 'scaled_reward_history' in result else reward
                    })
    
    # データフレーム化
    df_rewards = pd.DataFrame(reward_data)
    
    # 各次元の平均報酬を計算
    dim_rewards_summary = df_rewards.groupby(['Algorithm', 'Dimension'])['Reward'].agg(['mean', 'std', 'sum']).reset_index()
    
    # 可視化
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. 各次元の累積報酬（スケーリングなし）
    ax1 = axes[0, 0]
    without_scaling_data = dim_rewards_summary[dim_rewards_summary['Algorithm'] == 'WithoutScaling']
    ax1.bar(without_scaling_data['Dimension'], without_scaling_data['sum'], color='red', alpha=0.7)
    ax1.axvline(x=4.5, color='green', linestyle='--', label='Effective dims boundary')
    ax1.set_xlabel('Dimension')
    ax1.set_ylabel('Cumulative Reward')
    ax1.set_title(f'{func_name}: 各次元の累積報酬（スケーリングなし）')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. 各次元の累積報酬（スケーリングあり）
    ax2 = axes[0, 1]
    with_scaling_data = dim_rewards_summary[dim_rewards_summary['Algorithm'] == 'WithScaling']
    ax2.bar(with_scaling_data['Dimension'], with_scaling_data['sum'], color='blue', alpha=0.7)
    ax2.axvline(x=4.5, color='green', linestyle='--', label='Effective dims boundary')
    ax2.set_xlabel('Dimension')
    ax2.set_ylabel('Cumulative Reward')
    ax2.set_title(f'{func_name}: 各次元の累積報酬（スケーリングあり）')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. 報酬の時系列推移（有効次元の平均）
    ax3 = axes[1, 0]
    effective_dims = list(range(5))
    
    for alg_name in ['WithScaling', 'WithoutScaling']:
        alg_data = df_rewards[df_rewards['Algorithm'] == alg_name]
        effective_data = alg_data[alg_data['Dimension'].isin(effective_dims)]
        
        # イテレーションごとの有効次元の平均報酬
        iter_mean_rewards = effective_data.groupby('Iteration')['Reward'].mean()
        
        color = 'blue' if alg_name == 'WithScaling' else 'red'
        label = 'スケーリングあり' if alg_name == 'WithScaling' else 'スケーリングなし'
        ax3.plot(iter_mean_rewards.index, iter_mean_rewards.values, color=color, label=label, alpha=0.7)
    
    ax3.set_xlabel('Iteration')
    ax3.set_ylabel('Average Reward (Effective Dims)')
    ax3.set_title(f'{func_name}: 有効次元の平均報酬推移')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. スケーリングの効果（スケーリングあり版のスケール前後比較）
    ax4 = axes[1, 1]
    if 'scaled_reward_history' in results_dict['WithScaling'][0]:
        # スケーリング前後の報酬比較
        scaled_data = df_rewards[df_rewards['Algorithm'] == 'WithScaling']
        
        # 各次元のスケーリング前後の平均報酬
        dim_original = scaled_data.groupby('Dimension')['Reward'].mean()
        dim_scaled = scaled_data.groupby('Dimension')['Scaled_Reward'].mean()
        
        x = np.arange(len(dim_original))
        width = 0.35
        
        ax4.bar(x - width/2, dim_original.values, width, label='スケーリング前', color='orange', alpha=0.7)
        ax4.bar(x + width/2, dim_scaled.values, width, label='スケーリング後', color='blue', alpha=0.7)
        
        ax4.axvline(x=4.5, color='green', linestyle='--', label='Effective dims boundary')
        ax4.set_xlabel('Dimension')
        ax4.set_ylabel('Average Reward')
        ax4.set_title(f'{func_name}: スケーリング前後の報酬比較')
        ax4.legend()
        ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/{func_name}_reward_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    return df_rewards, dim_rewards_summary

print("報酬履歴分析関数の定義完了")

In [ ]:
# 可視化関数
def plot_comparison_results(results_dict, func_name, global_optimum, analytical_L_hat):
    """比較結果の可視化"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. 収束履歴の比較
    ax1 = axes[0, 0]
    colors = {'WithScaling': 'blue', 'WithoutScaling': 'red'}
    labels = {'WithScaling': f'スケーリングあり (L_hat={analytical_L_hat:.1f})', 'WithoutScaling': 'スケーリングなし'}
    
    for alg_name, results in results_dict.items():
        all_histories = [result['eval_history'] for result in results]
        histories_array = np.array(all_histories)
        
        mean_history = np.mean(histories_array, axis=0)
        std_history = np.std(histories_array, axis=0)
        iterations = np.arange(1, len(mean_history) + 1)
        
        ax1.plot(iterations, mean_history, color=colors[alg_name], 
                label=f'{labels[alg_name]}', linewidth=2)
        ax1.fill_between(iterations, mean_history - std_history, 
                        mean_history + std_history, color=colors[alg_name], alpha=0.2)
    
    ax1.axhline(y=global_optimum, color='green', linestyle='--', 
               label=f'Global optimum: {global_optimum:.2f}', linewidth=2)
    ax1.set_xlabel('Iterations')
    ax1.set_ylabel('Best Value Found')
    ax1.set_title(f'{func_name}: 収束履歴比較')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. 最終性能の比較
    ax2 = axes[0, 1]
    final_values = {}
    for alg_name, results in results_dict.items():
        final_values[alg_name] = [result['best_value'] for result in results]
    
    box_data = [final_values['WithScaling'], final_values['WithoutScaling']]
    box = ax2.boxplot(box_data, labels=['スケーリングあり', 'スケーリングなし'], patch_artist=True)
    box['boxes'][0].set_facecolor('lightblue')
    box['boxes'][1].set_facecolor('lightcoral')
    
    ax2.axhline(y=global_optimum, color='green', linestyle='--', 
               label=f'Global optimum: {global_optimum:.2f}', linewidth=2)
    ax2.set_ylabel('Final Best Value')
    ax2.set_title(f'{func_name}: 最終性能比較')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. 方向使用頻度の比較（スケーリングあり）
    ax3 = axes[1, 0]
    with_scaling_results = results_dict['WithScaling']
    if with_scaling_results[0]['theta_history']:
        all_final_theta = []
        for result in with_scaling_results:
            if result['theta_history']:
                final_theta = result['theta_history'][-1].abs().cpu().numpy()
                all_final_theta.append(final_theta)
        
        if all_final_theta:
            mean_theta = np.mean(all_final_theta, axis=0)
            std_theta = np.std(all_final_theta, axis=0)
            
            ax3.bar(range(len(mean_theta)), mean_theta, yerr=std_theta, 
                   capsize=5, color='lightblue', alpha=0.7)
            ax3.axvline(x=4.5, color='red', linestyle='--', 
                       label='Effective dims boundary', linewidth=2)
            ax3.set_xlabel('Dimension')
            ax3.set_ylabel('Absolute Theta Value')
            ax3.set_title(f'{func_name}: 方向使用頻度 (スケーリングあり)')
            ax3.legend()
            ax3.grid(True, alpha=0.3)
    
    # 4. 方向使用頻度の比較（スケーリングなし）
    ax4 = axes[1, 1]
    without_scaling_results = results_dict['WithoutScaling']
    if without_scaling_results[0]['theta_history']:
        all_final_theta = []
        for result in without_scaling_results:
            if result['theta_history']:
                final_theta = result['theta_history'][-1].abs().cpu().numpy()
                all_final_theta.append(final_theta)
        
        if all_final_theta:
            mean_theta = np.mean(all_final_theta, axis=0)
            std_theta = np.std(all_final_theta, axis=0)
            
            ax4.bar(range(len(mean_theta)), mean_theta, yerr=std_theta, 
                   capsize=5, color='lightcoral', alpha=0.7)
            ax4.axvline(x=4.5, color='red', linestyle='--', 
                       label='Effective dims boundary', linewidth=2)
            ax4.set_xlabel('Dimension')
            ax4.set_ylabel('Absolute Theta Value')
            ax4.set_title(f'{func_name}: 方向使用頻度 (スケーリングなし)')
            ax4.legend()
            ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/{func_name}_gradient_scaling_comparison.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    # 統計的要約の表示
    print(f"\n=== {func_name} 結果要約 ===")
    for alg_name, results in results_dict.items():
        final_values = [result['best_value'] for result in results]
        label = 'スケーリングあり' if alg_name == 'WithScaling' else 'スケーリングなし'
        print(f"{label}:")
        print(f"  平均: {np.mean(final_values):.6f}")
        print(f"  標準偏差: {np.std(final_values):.6f}")
        print(f"  最良値: {np.min(final_values):.6f}")
        print(f"  最悪値: {np.max(final_values):.6f}")

print("可視化関数の定義完了")

In [ ]:
# 実験の実行
all_results = {}
all_reward_dfs = {}
n_runs = 20

# 境界値
dim = 20
bounds = torch.tensor([[-5.0]*dim, [5.0]*dim], dtype=torch.float32)

for func_name, objective_function in test_functions.items():
    print(f"\n{'='*50}")
    print(f"実験: {func_name}")
    print(f"{'='*50}")
    
    # 解析的なL_hatを計算
    analytical_L_hat = compute_analytical_max_gradient(func_name, bounds, effective_dims=5)
    
    # 実験実行
    results = run_comparison_experiment(func_name, objective_function, analytical_L_hat, n_runs)
    all_results[func_name] = results
    
    # 結果の保存
    np.save(f'{output_dir}/{func_name}_gradient_scaling_results_fixed_L_hat.npy', results)
    
    # 基本的な可視化
    plot_comparison_results(results, func_name, global_optima[func_name], analytical_L_hat)
    
    # 報酬履歴の詳細分析
    df_rewards, dim_rewards_summary = analyze_reward_history(results, func_name)
    all_reward_dfs[func_name] = df_rewards
    
    # データフレームをCSVとして保存
    df_rewards.to_csv(f'{output_dir}/{func_name}_reward_history.csv', index=False)
    dim_rewards_summary.to_csv(f'{output_dir}/{func_name}_dimension_rewards_summary.csv', index=False)
    
    print(f"\n報酬履歴をCSVファイルに保存しました")

print("\n全ての実験が完了しました！")
print(f"結果は {output_dir} フォルダに保存されています。")

In [ ]:
# 全体の比較サマリー
print("\n" + "="*70)
print("全体結果サマリー: 固定L_hatスケーリングの効果検証")
print("="*70)
print("両アルゴリズムの共通設定:")
print("- 勾配ベース報酬（GPモデルの勾配の絶対値）")
print("- 0.5x arms（次元数の半分のアーム数）")
print("- coordinate_ratio=0.8")
print("違い: 解析的に求めた固定L_hatによるスケーリングの有無")
print("="*70)

for func_name in test_functions.keys():
    results = all_results[func_name]
    
    with_scaling_final = [result['best_value'] for result in results['WithScaling']]
    without_scaling_final = [result['best_value'] for result in results['WithoutScaling']]
    
    # 改善率の計算
    with_scaling_mean = np.mean(with_scaling_final)
    without_scaling_mean = np.mean(without_scaling_final)
    improvement_rate = (without_scaling_mean - with_scaling_mean) / abs(without_scaling_mean) * 100
    
    # 解析的L_hatの取得
    dim = 20
    bounds = torch.tensor([[-5.0]*dim, [5.0]*dim], dtype=torch.float32)
    analytical_L_hat = compute_analytical_max_gradient(func_name, bounds, effective_dims=5)
    
    print(f"\n{func_name}:")
    print(f"  解析的L_hat: {analytical_L_hat:.2f}")
    print(f"  スケーリングあり平均: {with_scaling_mean:.6f}")
    print(f"  スケーリングなし平均: {without_scaling_mean:.6f}")
    print(f"  スケーリングによる改善率: {improvement_rate:.2f}%")
    
    # 統計的有意差の検定（Mann-Whitney U test）
    from scipy.stats import mannwhitneyu
    statistic, p_value = mannwhitneyu(with_scaling_final, without_scaling_final, alternative='two-sided')
    print(f"  統計的有意差 (p-value): {p_value:.4f}")
    
    if p_value < 0.05:
        if with_scaling_mean < without_scaling_mean:
            print(f"  → 固定スケーリングありが統計的に有意に優秀")
        else:
            print(f"  → スケーリングなしが統計的に有意に優秀")
    else:
        print(f"  → 統計的有意差なし")
    
    # 報酬の統計情報
    df = all_reward_dfs[func_name]
    effective_dims_mask = df['Dimension'] < 5
    
    # 有効次元と非有効次元の報酬比較
    effective_rewards = df[effective_dims_mask].groupby('Algorithm')['Reward'].mean()
    non_effective_rewards = df[~effective_dims_mask].groupby('Algorithm')['Reward'].mean()
    
    print(f"  有効次元平均報酬 (スケーリングあり): {effective_rewards.get('WithScaling', 0):.3f}")
    print(f"  有効次元平均報酬 (スケーリングなし): {effective_rewards.get('WithoutScaling', 0):.3f}")
    print(f"  非有効次元平均報酬 (スケーリングあり): {non_effective_rewards.get('WithScaling', 0):.3f}")
    print(f"  非有効次元平均報酬 (スケーリングなし): {non_effective_rewards.get('WithoutScaling', 0):.3f}")

print("\n" + "="*70)
print("実験設定:")
print("- 20次元問題（有効次元: 0-4）")
print("- 各アルゴリズム20回独立実行")
print("- 300回評価")
print("- 固定L_hatスケーリング: 解析的に求めた最大勾配ノルムで正規化")
print("- 報酬履歴をCSVファイルとして保存")
print("="*70)